In [1]:
%%time 
from data.pair_feature import *
from data.original import item_pairs_train, item_pairs_test, item_info_train, item_info_test
from data.corpus_based import feature_nodes as corpus_based_feature_nodes
# from data.dummy import dummy_features, categoryID_shuffle_features
from data.item import get_item, show_item_pair
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import train_test_split
n_train = 3344613
n_test = 1315205
import pickle

CPU times: user 55.1 s, sys: 16.2 s, total: 1min 11s
Wall time: 1min 26s


In [2]:
def build_mxnet_header(model_names):
    header = ["line_num"]
    for name in model_names:
        header.extend('mxnet_{}_batch_{}_sim'.format(name, x) for x in  ['min', 'max', 'summean', 'mean'])
    return header

image_mxnet_features_train = pd.read_csv('data/data_files/image_mxnet_feature_train_bn.csv', names=build_mxnet_header(['bn']), index_col='line_num').sort_index()
image_mxnet_features_test = pd.read_csv('data/data_files/image_mxnet_feature_test_bn.csv', names=build_mxnet_header(['bn']), index_col='line_num').sort_index()

In [3]:
from feature.image_histogram_feature import header as image_histogram_feature_header
image_histogram_features_train = pd.read_csv('data/data_files/image_histogram_feature_train.csv', names = image_histogram_feature_header, index_col='index').sort_index()
image_histogram_features_test = pd.read_csv('data/data_files/image_histogram_feature_test.csv', names = image_histogram_feature_header, index_col='index').sort_index()

In [4]:
from feature.image_hash_feature import headers as image_hash_feature_headers

image_hash_features_train = pd.read_csv('data/data_files/image_hash_feature_train.csv',
                                       names=image_hash_feature_headers, index_col='index').sort_index()
image_hash_features_test = pd.read_csv('data/data_files/image_hash_feature_test.csv',
                                       names=image_hash_feature_headers, index_col='index').sort_index()

In [5]:
image_features_train = pd.concat((image_histogram_features_train, image_hash_features_train, image_mxnet_features_train), axis=1)
image_features_test = pd.concat((image_histogram_features_test, image_hash_features_test, image_mxnet_features_test), axis=1)

In [6]:
corpus_based_features = []
for node in corpus_based_feature_nodes:
    corpus_based_features.append(node.get_data())
corpus_based_features = pd.concat(corpus_based_features, axis=1)

In [7]:
corpus_based_features.shape

(4035592, 110)

In [8]:
def split_feats(feats):
    n_train = 2991396
    feats_train = feats[:n_train]
    feats_test = feats[n_train:]
    feats_test.index = simple_features_test.index.copy()
    return feats_train, feats_test

In [9]:
corpus_based_features_train, corpus_based_features_test = split_feats(corpus_based_features)
# dummy_features_train, dummy_features_test = split_feats(dummy_features)
# categoryID_shuffle_features_train, categoryID_shuffle_features_test = split_feats(categoryID_shuffle_features)

In [10]:
print(simple_features_train.shape[1])
print(aggregation_features_train.shape[1])
print(title_features_train.shape[1])
print(description_features_train.shape[1])
print(image_features_train.shape[1])
print(corpus_based_features_train.shape[1])
# print(categoryID_shuffle_features_train.shape[1])

26
20
4
10
44
110


In [11]:
features_train = pd.concat((
        simple_features_train,
        aggregation_features_train,
        title_features_train, description_features_train,
        ncd_features_train,
        image_features_train,
        corpus_based_features_train,
#         dummy_features_train,
#         categoryID_shuffle_features_train,
    ), axis=1)
features_test = pd.concat((
        simple_features_test,
        aggregation_features_test,
        title_features_test, description_features_test,
        ncd_features_test,
        image_features_test, 
        corpus_based_features_test,
#         dummy_features_test,
#         categoryID_shuffle_features_test
    ), axis=1)

In [12]:
print(features_train.shape)
print(features_test.shape) 

(2991396, 218)
(1044196, 218)


In [14]:
weight_map = {
    1: 3,
    2: 1,
    3: 1
}

weight = np.empty(item_pairs_train.shape[0])
generationMethod = item_pairs_train.generationMethod.values
for i in [1,2,3]:
    weight[generationMethod==i] = weight_map[i]

In [70]:
%%time
X_train, X_test, y_train, y_test, I_train, I_test =\
train_test_split(features_train.values, item_pairs_train.isDuplicate.values, np.arange(features_train.shape[0]),
                 test_size=0.2, random_state=42)
weight_train = weight[I_train]

CPU times: user 6min 58s, sys: 26.7 s, total: 7min 24s
Wall time: 7min 25s


In [98]:
model = XGBClassifier(learning_rate=0.1, max_depth=10, subsample=1, colsample_bytree=0.8, n_estimators=260,
                      min_child_weight = 1,
                      nthread=32)

In [99]:
model.fit(X_train, y_train, eval_metric='auc', eval_set=[(X_test, y_test)],  early_stopping_rounds=20)

[0]	validation_0-auc:0.931742
Will train until validation_0-auc hasn't improved in 20 rounds.
[1]	validation_0-auc:0.935044
[2]	validation_0-auc:0.93773
[3]	validation_0-auc:0.939017
[4]	validation_0-auc:0.93991
[5]	validation_0-auc:0.940911
[6]	validation_0-auc:0.941959
[7]	validation_0-auc:0.94302
[8]	validation_0-auc:0.943587
[9]	validation_0-auc:0.9444
[10]	validation_0-auc:0.945049
[11]	validation_0-auc:0.945653
[12]	validation_0-auc:0.946107
[13]	validation_0-auc:0.946582
[14]	validation_0-auc:0.947158
[15]	validation_0-auc:0.947623
[16]	validation_0-auc:0.948034
[17]	validation_0-auc:0.948471
[18]	validation_0-auc:0.948807
[19]	validation_0-auc:0.949265
[20]	validation_0-auc:0.949633
[21]	validation_0-auc:0.949993
[22]	validation_0-auc:0.950287
[23]	validation_0-auc:0.950631
[24]	validation_0-auc:0.950849
[25]	validation_0-auc:0.951172
[26]	validation_0-auc:0.951419
[27]	validation_0-auc:0.951655
[28]	validation_0-auc:0.951976
[29]	validation_0-auc:0.952282
[30]	validation_0-auc

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=260, nthread=32,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [100]:
%time pred_test = model.predict_proba(X_test)[:,1]

CPU times: user 1min 3s, sys: 1.34 s, total: 1min 4s
Wall time: 30.4 s


In [101]:
eval_set = item_pairs_train.iloc[I_test]
eval_set['prob_predicted'] = pred_test

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [102]:
from bokeh.io import output_notebook, show
from bokeh.charts import Histogram
output_notebook()

Loading BokehJS ...

In [103]:
from ml_metrics import auc

In [104]:
eval_set.groupby('isDuplicate')['prob_predicted'].mean()

isDuplicate
0    0.125419
1    0.825876
Name: prob_predicted, dtype: float32

In [105]:
eval_set.groupby('isDuplicate')['prob_predicted'].std()

isDuplicate
0    0.198409
1    0.247845
Name: prob_predicted, dtype: float32

In [106]:
from data.dummy import mergeLeftInOrder2

In [107]:
eval_set = mergeLeftInOrder2(eval_set, item_info_train[['categoryID']], left_on='itemID_1', right_index=True)

In [108]:
auc_by_category = eval_set.groupby('categoryID')[['isDuplicate', 'prob_predicted']].apply(lambda x: auc(x['isDuplicate'].values, x['prob_predicted'].values))

In [109]:
auc_by_category.order()

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


categoryID
112    0.858464
85     0.903141
33     0.913381
111    0.913763
101    0.927559
105    0.930247
31     0.932722
99     0.934950
10     0.940427
23     0.942282
97     0.944655
19     0.947516
26     0.948816
98     0.949131
34     0.949929
25     0.950278
32     0.951069
115    0.951840
42     0.951955
86     0.954261
21     0.957213
40     0.958290
24     0.958903
87     0.959818
84     0.960464
39     0.960946
96     0.963323
102    0.966269
29     0.966634
38     0.966994
36     0.967149
90     0.967512
83     0.968200
27     0.969401
28     0.971950
94     0.974298
114    0.974501
20     0.974637
89     0.976800
9      0.978176
106    0.980815
88     0.981223
30     0.982920
93     0.983322
81     0.984331
14     0.985742
82     0.987740
92     0.988143
11     0.989218
91     0.989520
116    0.993231
dtype: float64

In [110]:
from bokeh.charts import Bar

In [111]:
show(Bar(auc_by_category.order()))

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


In [112]:
eval_set.shape[0]

598280

In [113]:
p = Histogram(eval_set.head(5000), values='prob_predicted', color='isDuplicate', bins=30, legend=True, plot_width=1000)
show(p)

In [114]:
feature_importance = pd.DataFrame({'feature': features_train.columns,
              'importance': model.feature_importances_}).sort('importance', ascending=False)

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [115]:
from bokeh.charts import Bar
from bokeh.charts.attributes import CatAttr

In [116]:
feature_importance.head(50)

,feature,importance
27,categoryID_price_mean,0.020427
26,categoryID_freq,0.019466
180,title_word_1_2gram_dtm_0_predict_log_price__1,0.016664
181,title_word_1_2gram_dtm_0_predict_log_price__2,0.016589
28,categoryID_price_median,0.016116
4,description_length_max,0.014382
5,description_length_min,0.014361
107,mxnet_bn_batch_mean_sim,0.014119
185,title_description_dtm_0_predict_log_price__2,0.013636
11,price_diff,0.013534


In [126]:
feature_importance.tail(50)

,feature,importance
195,description_sentence_word_dtm_0__binary_tf__co...,0.001815
165,title_word_dtm_0_1__max_disjoint_idf_max,0.001793
207,description_sentence_word_dtm_1__binary_tf__co...,0.001766
101,image_whash_hamming_1_min,0.001723
111,title_word_dtm_0__binary_tfidf__cosine,0.001707
85,image_phash_hamming_0_max,0.001696
167,title_word_dtm_1_1__max_disjoint_idf_max,0.001664
19,same_metroID,0.001654
146,title_word_2gram_dtm_0_1__tfidf__cosine,0.001632
122,title_word_dtm_1_1__tfidf__cosine,0.001616


In [128]:
corpus_based_features.columns

Index(['title_word_dtm_0__tf__cosine', 'title_word_dtm_0__binary_tf__cosine',
       'title_word_dtm_0__tfidf__cosine',
       'title_word_dtm_0__binary_tfidf__cosine',
       'title_word_dtm_1__tf__cosine', 'title_word_dtm_1__binary_tf__cosine',
       'title_word_dtm_1__tfidf__cosine',
       'title_word_dtm_1__binary_tfidf__cosine',
       'title_word_dtm_0_1__tf__cosine',
       'title_word_dtm_0_1__binary_tf__cosine',
       ...
       'description_sentence_word_dtm_1__binary_tf__cosine_similarity__0_mean',
       'description_sentence_word_dtm_1__binary_tf__cosine_similarity__1_min',
       'description_sentence_word_dtm_1__binary_tf__cosine_similarity__1_max',
       'description_sentence_word_dtm_1__binary_tf__cosine_similarity__1_mean',
       'description_sentence_word_dtm_1_1__binary_tf__cosine_similarity__0_min',
       'description_sentence_word_dtm_1_1__binary_tf__cosine_similarity__0_max',
       'description_sentence_word_dtm_1_1__binary_tf__cosine_similarity__0_mean',


In [117]:
item_info_train.shape[0] + item_info_test.shape[0]

4659818

In [118]:
bst = model.booster()
bst.feature_names = features_train.columns.tolist()
print(pd.Series(bst.get_score(importance_type='gain')).sort_values(ascending=False).head(50))
print(pd.Series(bst.get_score(importance_type='cover')).sort_values(ascending=False).head(50))

image_phash_hamming_0_min                         2260.781939
mxnet_bn_batch_max_sim                            2107.969613
title_word_dtm_0_1__binary_tfidf__cosine           719.205869
title_word_dtm_0_1__tfidf__cosine                  477.213628
title_word_dtm_0__binary_tfidf__cosine             334.473571
image_phash_hamming_0_mean                         265.589005
price_diff_ratio                                   229.616435
image_dhash_jaccard                                169.673189
title_word_dtm_0_1__max_disjoint_idf_min           158.088462
images_count_diff_ratio                            151.324306
categoryID_price_std                               123.680295
attrsJSON_item_jaccard                             115.896578
mxnet_bn_batch_summean_sim                         113.475819
title_word_2gram_dtm_0__max_disjoint_idf_min       106.156270
categoryID_price_mean                              100.350208
image_phash_jaccard                                 99.663007
descript

In [119]:
feature_importance.set_index('feature').loc[corpus_based_features.columns].sort('importance', ascending=False)

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,importance
title_word_1_2gram_dtm_0_predict_log_price__1,0.016664
title_word_1_2gram_dtm_0_predict_log_price__2,0.016589
title_description_dtm_0_predict_log_price__2,0.013636
title_description_dtm_0_predict_log_price__1,0.012460
description_word_dtm_0__max_disjoint_idf_min,0.008756
description_word_lsa_0_1_1,0.006974
title_word_2gram_dtm_0__max_disjoint_idf_min,0.006856
description_word_nmf_0_1_1,0.006759
description_word_dtm_0__max_disjoint_idf_max,0.006539
title_word_dtm_0__max_disjoint_idf_min,0.006480


In [120]:
show(Bar(feature_importance.head(50), label=CatAttr(columns=['feature'], sort=False), values='importance',
        plot_width=1000))

In [122]:
import pickle
pickle.dump(eval_set, open(model_name+'.eval_set.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(model, open(model_name+'.eval.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [123]:
%%time
model.fit(features_train, item_pairs_train.isDuplicate, sample_weight=weight)
prob_test = model.predict_proba(features_test)[:,1]

pickle.dump(model, open(model_name+'.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pd.DataFrame({'id': item_pairs_test['id'], 'probability': prob_test}).to_csv(model_name+'.csv', index=False)

CPU times: user 19h 3min 4s, sys: 58.7 s, total: 19h 4min 3s
Wall time: 43min 50s


In [15]:
%%time
model_name = 'xgboost.38.tuning.2'
model = XGBClassifier(learning_rate=0.05, max_depth=10, subsample=0.8, colsample_bytree=0.8, n_estimators=300,
                      min_child_weight = 1,
                      nthread=32)
model.fit(features_train, item_pairs_train.isDuplicate, sample_weight=weight)
prob_test = model.predict_proba(features_test)[:,1]

pickle.dump(model, open(model_name+'.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pd.DataFrame({'id': item_pairs_test['id'], 'probability': prob_test}).to_csv(model_name+'.csv', index=False)

NameError: name 'pickle' is not defined

In [18]:
%%time
model_name = 'xgboost.38.tuning.3'
model = XGBClassifier(learning_rate=0.05, max_depth=10, subsample=0.8, colsample_bytree=0.8, n_estimators=500,
                      min_child_weight = 1,
                      nthread=32)
model.fit(features_train, item_pairs_train.isDuplicate, sample_weight=weight)
prob_test = model.predict_proba(features_test)[:,1]

pickle.dump(model, open(model_name+'.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pd.DataFrame({'id': item_pairs_test['id'], 'probability': prob_test}).to_csv(model_name+'.csv', index=False)

CPU times: user 1d 14h 1min 16s, sys: 1min 36s, total: 1d 14h 2min 53s
Wall time: 1h 27min 13s


In [19]:
%%time
model_name = 'xgboost.38.tuning.4'
model = XGBClassifier(learning_rate=0.05, max_depth=10, subsample=0.6, colsample_bytree=0.6, n_estimators=500,
                      min_child_weight = 1,
                      nthread=32)
model.fit(features_train, item_pairs_train.isDuplicate, sample_weight=weight)
prob_test = model.predict_proba(features_test)[:,1]

pickle.dump(model, open(model_name+'.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pd.DataFrame({'id': item_pairs_test['id'], 'probability': prob_test}).to_csv(model_name+'.csv', index=False)

CPU times: user 1d 4h 34min 41s, sys: 1min 21s, total: 1d 4h 36min 2s
Wall time: 1h 7min 57s


In [20]:
%%time
model_name = 'xgboost.38.tuning.5'
model = XGBClassifier(learning_rate=0.05, max_depth=10, subsample=1, colsample_bytree=1, n_estimators=500,
                      min_child_weight = 1,
                      nthread=32)
model.fit(features_train, item_pairs_train.isDuplicate, sample_weight=weight)
prob_test = model.predict_proba(features_test)[:,1]

pickle.dump(model, open(model_name+'.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pd.DataFrame({'id': item_pairs_test['id'], 'probability': prob_test}).to_csv(model_name+'.csv', index=False)

CPU times: user 1d 22h 19min 22s, sys: 1min 48s, total: 1d 22h 21min 10s
Wall time: 1h 43min 47s


In [21]:
%%time
model_name = 'xgboost.38.tuning.6'
model = XGBClassifier(learning_rate=0.05, max_depth=15, subsample=0.8, colsample_bytree=0.8, n_estimators=500,
                      min_child_weight = 1,
                      nthread=32)
model.fit(features_train, item_pairs_train.isDuplicate, sample_weight=weight)
prob_test = model.predict_proba(features_test)[:,1]

pickle.dump(model, open(model_name+'.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pd.DataFrame({'id': item_pairs_test['id'], 'probability': prob_test}).to_csv(model_name+'.csv', index=False)

CPU times: user 2d 14h 34min 13s, sys: 2min 21s, total: 2d 14h 36min 34s
Wall time: 2h 21min 28s


In [15]:
%%time
model_name = 'xgboost.38.tuning.7'
model = XGBClassifier(learning_rate=0.05, max_depth=10, subsample=0.8, colsample_bytree=0.8, n_estimators=1000,
                      min_child_weight = 1,
                      nthread=32)
model.fit(features_train, item_pairs_train.isDuplicate, sample_weight=weight)
prob_test = model.predict_proba(features_test)[:,1]

pickle.dump(model, open(model_name+'.pickle', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pd.DataFrame({'id': item_pairs_test['id'], 'probability': prob_test}).to_csv(model_name+'.csv', index=False)

CPU times: user 3d 48min 44s, sys: 2min 19s, total: 3d 51min 4s
Wall time: 2h 43min 55s
